# 🔬 Projet corrigé : Enquête scientifique par analyse d’ADN

Ce carnet contient une solution possible à l'enquête ADN avec des exemples de fonctions, d'analyses et de visualisations.


## 🧭 1. Planification

Étapes prévues :
1. Charger et explorer les données
2. Créer une fonction pour comparer deux séquences malgré les `?`
3. Calculer les pourcentages de correspondance pour chaque suspect et chaque locus
4. Créer un tableau synthèse
5. Générer un diagramme à barres avec barres d’erreur
6. Effectuer une régression linéaire entre deux loci
7. Interpréter les résultats


## 🧼 2. Chargement et nettoyage des données

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

df = pd.read_csv("suspects.csv")
df.head()

## 🧮 3. Calculs de correspondance

In [ ]:
def comparer_sequences(seq1, seq2):
    correspondances = 0
    valides = 0
    for a, b in zip(seq1, seq2):
        if a != '?' and b != '?':
            valides += 1
            if a == b:
                correspondances += 1
    if valides == 0:
        return np.nan
    return (correspondances / valides) * 100


In [ ]:
def analyser_suspects(df):
    loci = ['Locus_1', 'Locus_2', 'Locus_3']
    reference = df[df["Nom"] == "Crime"].iloc[0]
    resultats = []

    for i, row in df.iterrows():
        if row["Nom"] == "Crime":
            continue
        scores = []
        for locus in loci:
            score = comparer_sequences(row[locus], reference[locus])
            scores.append(score)
        moyenne = np.nanmean(scores)
        resultats.append({
            "Nom": row["Nom"],
            "Locus_1": scores[0],
            "Locus_2": scores[1],
            "Locus_3": scores[2],
            "Moyenne": moyenne
        })
    return pd.DataFrame(resultats)

df_scores = analyser_suspects(df)
df_scores

## 📊 4. Visualisation des résultats

In [ ]:
def graphique_correspondances(df_scores):
    labels = df_scores["Nom"]
    x = np.arange(len(labels))
    width = 0.2

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(x - width, df_scores["Locus_1"], width, label="Locus 1")
    ax.bar(x, df_scores["Locus_2"], width, label="Locus 2")
    ax.bar(x + width, df_scores["Locus_3"], width, label="Locus 3")

    for i in range(len(labels)):
        for j, col in enumerate(["Locus_1", "Locus_2", "Locus_3"]):
            ax.errorbar(x[i] + (j - 1) * width, df_scores[col][i], yerr=5, fmt='none', ecolor='black')

    ax.set_ylabel('Pourcentage de correspondance')
    ax.set_title('Correspondance ADN par locus')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()
    plt.tight_layout()
    plt.show()

graphique_correspondances(df_scores)

## 📈 5. Régression linéaire

In [ ]:
x = df_scores["Locus_1"]
y = df_scores["Locus_2"]
res = stats.linregress(x, y)

plt.figure(figsize=(6, 5))
plt.scatter(x, y)
plt.plot(x, res.intercept + res.slope * x, 'r', label=f'y={res.slope:.2f}x+{res.intercept:.2f}')
plt.xlabel("Locus 1")
plt.ylabel("Locus 2")
plt.title("Régression linéaire entre Locus 1 et Locus 2")
plt.legend()
plt.grid()
plt.show()

print(f"Pente : {res.slope:.2f} ± {res.stderr:.2f}")
print(f"R² : {res.rvalue**2:.2f}")

## 🧠 6. Interprétation

- Le ou les suspects avec les taux les plus élevés sont les plus probables.
- L’analyse montre une corrélation modérée entre les loci 1 et 2.
- Les incertitudes biologiques (caractères manquants) diminuent la précision.
- On pourrait améliorer l’analyse avec plus de loci ou d’outils bioinformatiques.


## 🧪 Bonus : Limites biologiques

- Les mutations naturelles ou les erreurs de séquençage peuvent fausser les résultats.
- La contamination ou la dégradation des échantillons sont des facteurs à considérer.
